In [1]:
from IPython.display import HTML

HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')


In [2]:
import pandas as pd
from xbbg import blp
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime as dt
from datetime import timedelta as td
import statsmodels.api as sm
import re
import matplotlib.image as image
from ipywidgets import interact, IntSlider, Checkbox, Dropdown, Output, HBox, VBox, interactive, interactive_output, ToggleButton,Text, Button, DatePicker, IntText, ToggleButtons, RadioButtons,SelectMultiple, FloatText
from IPython.display import display, clear_output
import itertools
from scipy import stats
from scipy.optimize import minimize 
from scipy.special import ndtr
import warnings
spot_library = {'eur_6':['EUSA', ' Curncy'],
                 'eur': ['EESWE', ' Curncy'], #ois
                 'usd': ['USOSFR', ' Curncy'], #ois
                 'gbp': ['BPSWS', ' Curncy'],#ois
                 'chf': ['SFSNT', ' Curncy'],#ois
                 'sek': ['SKSW', ' Curncy'],#irs
                 'nok': ['NKSW', ' Curncy'],#irs
                 'hkd': ['HDSW', ' Curncy'],#irs
                 'czk': ['CKSW', ' Curncy'],#irs
                 'pln': ['PZSW', ' Curncy'],#irs
                 'ils':['ISSW', ' Curncy'], #irs
                 'cad':['CDSW', ' Curncy'], #irs
                 'jpy':['JYSO', ' Curncy'],#ois
                 'aud': ['ADSW', ' Curncy'],#irs
                'sgd':['SDSW', ' Curncy'],#irs
                'krw': ['KWSWNI', ' Curncy'],#irs
                'zar': ['SASW', ' Curncy'],#irs
                'nzd': ['NDSW', ' Curncy'],#irs
                'mxn': ['MPSW', ' Curncy']} #irs

forward_library = {'eur_6': ['EUSA', ' Curncy'], #Z
                 'eur': ['S0514FS ', ' BLC Curncy'], 
                 'usd': ['S0490FS ', ' BLC Curncy'], 
                 'gbp': ['S0141FS ', ' BLC Curncy'],
                 'chf': ['S0234FS ', ' BLC Curncy'],
                 'sek': ['SD0020FS ', ' BLC Curncy'],
                 'nok': ['SD0313FS ', ' BLC Curncy'],
                 'hkd': ['HDFS', ' Curncy'],#Z
                 'czk': ['S0320FS ', ' BLC Curncy'],
                 'pln': ['S0323FS ', ' BLC Curncy'],
                 'ils': ['ISFS', ' Curncy'], #Z
                 'cad': ['CDFS', ' Curncy'], #Z
                 'jpy': ['S0195FS ', ' BLC Curncy'],
                 'aud': ['SD0302FS ', ' BLC Curncy'],
                'sgd': ['SDFS', ' Curncy'],#Z
                'krw': ['S0205FS ', ' BLC Curncy'],
                'zar': ['SAFS', ' Curncy'],#
                'nzd': ['SD0015FS ', ' BLC Curncy'],
                'mxn': ['SD0083FS ', ' BLC Curncy']} 


In [3]:

def cut(some):
    x = 0
    y = some[0]
    while y.isdigit():
        y=some[x]
        x+=1
    return some[:x],some[x:]


def f(tenor):
    x = re.findall(r'\d+',tenor)[0]
    num = ''
    if 'm' in tenor.lower():
        if int(x) // 12 >0:
            num+='1'
        num+=chr(64+(int(x)%12))
    else:
        if len(x) == 1:
            num+='0'
        num +=x
    return num

def t(tenor):
    x = re.findall(r'\d+',tenor)[0]
    num = ''
    if 'm' in tenor.lower():
        if int(x) // 12 >0:
            num+='1'
        else:
            num+='0'
        num+=chr(64+(int(x)%12))
    else:
        if len(x) == 1:
            num+='0'
        num +=x
    return num


def spot_ticker(dex,tenor):
    dex = dex.lower()
    if dex.lower() == 'mxn':
        y = int(tenor[:-1]) * 13
        num = f'{y//12}{chr(64+(y%12))}'
    else:
        num = tenor[:-1]
    return f'{spot_library[dex][0]}{num}{spot_library[dex][1]}'

def forward_ticker(dex,fwd):
    dex = dex.lower()
    fwd = fwd.lower()
    old = ['eur_6','hkd','ils','cad','sgd','zar']
    
    if cut(fwd)[0] == '0y':
        return spot_ticker(dex,cut(fwd)[1])
    elif dex == 'eur_6':
        F,T = f(cut(fwd)[0]),t(cut(fwd)[1])
        return f'{forward_library[dex][0]}{F}{T}{forward_library[dex][1]}'
    elif dex in old:
        F,T = t(cut(fwd)[0]),t(cut(fwd)[1])
        return f'{forward_library[dex][0]}{F}{T}{forward_library[dex][1]}'
    else:
        return f'{forward_library[dex][0]}{fwd.upper()}{forward_library[dex][1]}'
    
def structure(dex,structure,start, end = 'today',bps = True):
    f_fly = structure.count('/') == 2
    f_crv = structure.count('/') == 1  
    fly = structure.count('s') == 3
    crv = structure.count('s') == 2
    out = max(2 -sum([i.isalpha() for i in structure]),0)
    
    if f_fly or f_crv:
        legs = [forward_ticker(dex,i) for i in structure.split('/')]
    elif fly or crv:
        legs = [spot_ticker(dex,i) for i in [i+'Y' for i in structure.split('s') if i.isdigit()]]
    else:
        legs = forward_ticker(dex,('0Y'*out) + structure)
        
    df = blp.bdh(legs, flds='px_last', start_date=start,end_date=end)  *(100 if bps else 1)
    s  = pd.DataFrame({})
    if f_fly or fly:
        x = (2 * df.iloc[:,1]) - (df.iloc[:,0] + df.iloc[:,2])
    elif f_crv or crv:
        x = df.iloc[:,1] - df.iloc[:,0]
    else:
        x = df.iloc[:,0]
        
    s[f'{dex.upper()} {structure}'] = x
    return s

def threeM_roll(structure): # only valid for >=1y f
    fds = [int(i.split('y')[0])*12 -3 for i in structure.split('/')]
    act = [int(i.split('y')[1]) for i in structure.split('/')]
    roll_to = [f'{i}M{j}Y' for i,j in zip(fds,act)]
    new_structure =''
    for i in range(len(roll_to)):
        if i !=0:
            new_structure += '/'    
        new_structure += roll_to[i]
    return new_structure
    

### MOVEMENTS ALONG THE FORWARD CURVES


In [4]:
chg_dict = {'1W':5,'2W':10,'1M':20,'2M':40,'3M':60}

In [5]:
chg_dict = {'1W':5,'2W':10,'1M':20,'2M':40,'3M':60}
output1 = Output()

COUNTRIES_W = SelectMultiple(options = [i.upper() for i in spot_library.keys()],
                              value = ['EUR', 'USD', 'GBP','CHF','AUD','CAD'],
                             description = 'Curves')

CHANGE_OVER_W = Dropdown(options = chg_dict.keys(), description = 'Change over') 
END_W = DatePicker(value = dt.today().date(), description = 'As of' )

SHOW_FWD_B = Button(description = 'Chart Moves')

display(COUNTRIES_W,CHANGE_OVER_W,END_W)
display(SHOW_FWD_B, output1)

def on_button_clicked1(b):
    with output1:
        clear_output()
        
        counter = 0
        tenors = ['0Y1Y','1Y1Y','2Y1Y','3Y1Y','4Y1Y','5Y1Y','6Y1Y','7Y3Y','10Y2Y','15Y5Y','20Y5Y','25Y5Y']
        fc = pd.DataFrame({}, columns = [i.upper() for i in COUNTRIES_W.value], index = tenors)
        start = END_W.value - td(days = 2 * chg_dict[CHANGE_OVER_W.value])
        
        df = None
        for i in COUNTRIES_W.value:
            
            denom = len(tenors) * len(COUNTRIES_W.value) 
            data_loaded = round(100 * counter/denom,2) 
            print(f"Loading data...({data_loaded}% done)")
            
            for j in tenors:
                if isinstance(df,pd.core.frame.DataFrame):
                    h = structure(i,j,start=start,end=END_W.value)
                    df = df.join(h,how = 'inner')
                else:
                    df = structure(i,j,start=start,end=END_W.value)
                counter+=1
            clear_output()
        
        _from, _to = df.index[-chg_dict[CHANGE_OVER_W.value]], df.index[-1]
        change = df.loc[_to] - df.loc[_from]
        
        for i in change.index:
            col,row = i.split(' ')
            fc.at[row, col] = change[i]
        
        fig,ax = plt.subplots(figsize = (18,8))
        for i in fc.columns:
            ax.plot(fc[i],label = i, marker = 'D')
        
        ax.legend()
        ax.set_title(f"FORWARD CURVES, {CHANGE_OVER_W.value} CHANGE AS OF {END_W.value.strftime('%d %B')}".upper())
        ax.hlines(y=0,linestyles='--',xmin = 0,xmax = 11)
        ax.set_ylabel('bps')
        plt.show()

SHOW_FWD_B.on_click(on_button_clicked1)

SelectMultiple(description='Curves', index=(1, 2, 3, 4, 13, 11), options=('EUR_6', 'EUR', 'USD', 'GBP', 'CHF',…

Dropdown(description='Change over', options=('1W', '2W', '1M', '2M', '3M'), value='1W')

DatePicker(value=datetime.date(2023, 9, 25), description='As of')

Button(description='Chart Moves', style=ButtonStyle())

Output()

### STRUCTURES AGAINST A REFERENCE
The following chart plots a cross-sectional relation between a reference tenor move versus the move in the desired structure.


In [6]:
df = None
regressor = None
regressand = None
data_ready = False
countries = [i.upper() for i in spot_library.keys()]

In [7]:
TARGET_STR_W = Text(description = 'Regress')
REFERENCE_W = Text(description = 'on')
CHANGE_OVER1_W = Dropdown(options = chg_dict.keys(), description = 'Change over') 
END1_W = DatePicker(value = dt.today().date(), description = 'As of' )
METRICS_W = ToggleButtons(options = ['Scatter', 'Model Diagnostics', 'Dislocations','Daily Vol.'])

output2 = Output()

RUN_REG_B = Button(description = 'Run Regression')
LOAD_DATA_B = Button(description = 'Load Data')

display(HBox([TARGET_STR_W,REFERENCE_W]),CHANGE_OVER1_W,END1_W, METRICS_W)
display(LOAD_DATA_B,RUN_REG_B,output2)

def on_button_clicked2(b):    
    global df
    global regressor
    global regressand
    global data_ready
    
    with output2:
        clear_output()
        counter = 0
        regressor, regressand = REFERENCE_W.value,TARGET_STR_W.value 
        start = dt(END1_W.value.year -1,END1_W.value.month,END1_W.value.day)
        for c in countries:
            for s in [REFERENCE_W.value,TARGET_STR_W.value]:
                data_loaded = round(50 * counter/len(countries),2) 
                print(f"Loading data...({data_loaded}% done)")
                counter += 1
                try:
                    if isinstance(df,pd.core.frame.DataFrame):
                        h = structure(c,s,start=start,end=END1_W.value)
                        df = df.join(h,how = 'inner')
                    else:
                        df= structure(c,s,start=start,end=END1_W.value)
                except:
                    pass
                clear_output()
        print("Data loaded")
        data_ready = True
        
    
def on_button_clicked3(b):
    with output2:
        clear_output()
        study = pd.DataFrame({}, index = countries, columns = [REFERENCE_W.value,TARGET_STR_W.value])
        
        
        x_and_y = (regressor == REFERENCE_W.value) & (regressand == TARGET_STR_W.value)
        
        if not data_ready:
            print('Please load the data before beginning analysis')
            
        elif not x_and_y:
            print(f'Data is loaded for different structure(s). Please press load data again for the\n structures you have entered')
        
        else:
            changes = df.loc[df.index[-1]] - df.loc[df.index[-chg_dict[CHANGE_OVER1_W.value]]] 
            for s in study.columns:
                for c in study.index:
                    try:
                        study.at[c,s] = changes[f"{c} {s}"]
                    except:
                        study.drop(index=c, inplace = True)
        
            study.dropna(inplace = True)
            study = study.astype(float)
            X = sm.add_constant(study[REFERENCE_W.value])
            model = sm.OLS(study[TARGET_STR_W.value],X).fit(cov_type = 'HC3')
            cons,slp = model.params
            xs = np.linspace(study[REFERENCE_W.value].min(), study[REFERENCE_W.value].max(),100)
            line = cons + slp*xs
            
            if METRICS_W.value == 'Scatter':
                fig,ax = plt.subplots(figsize = (15,8))
                ax.scatter(study[REFERENCE_W.value],study[TARGET_STR_W.value],marker = 'D')
                ax.plot(xs,line, color = 'r')
                ax.set_title(f"{TARGET_STR_W.value} VERSUS {REFERENCE_W.value}, {CHANGE_OVER1_W.value} CHANGE AS OF {END1_W.value.strftime('%d %B').upper()}")
                ax.set_ylabel(TARGET_STR_W.value)
                ax.set_xlabel(REFERENCE_W.value)
                for i in study.index:
                    ax.annotate(i,(study.at[i,REFERENCE_W.value],study.at[i,TARGET_STR_W.value]))
                plt.show()
                
            elif METRICS_W.value == 'Dislocations':
                display(pd.DataFrame({'DISLOCATIONS' :model.resid.values},index = model.resid.index).style.format('{:.2f}'))
            
            elif METRICS_W.value == 'Daily Vol.':
                cols = [i for i in df.columns if TARGET_STR_W.value in i]
                cols = [i for i in cols if i.split()[0] in study.index]
                vols = df[cols].diff().std()
                display(pd.DataFrame({'DAILY VOL' : vols.values},index = vols.index).style.format('{:.2f}'))
                
                
            else:
                display(model.summary())
            
            
           
        
LOAD_DATA_B.on_click(on_button_clicked2)
RUN_REG_B.on_click(on_button_clicked3)

Dropdown(description='Change over', options=('1W', '2W', '1M', '2M', '3M'), value='1W')

DatePicker(value=datetime.date(2023, 9, 25), description='As of')

ToggleButtons(options=('Scatter', 'Model Diagnostics', 'Dislocations', 'Daily Vol.'), value='Scatter')

Button(description='Load Data', style=ButtonStyle())

Button(description='Run Regression', style=ButtonStyle())

Output()